In [ ]:
!pip install pandas
!pip install windrose
!pip install matplotlib
!pip install selenium=3.141

# IMPORTANDO BIBLIOTECAS E FUNÇÕES AUXILIARES

In [25]:
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from windrose import WindroseAxes           # https://windrose.readthedocs.io/en/latest/usage.html
import matplotlib.ticker as tkr
from Rdita_Atrs_2024 import *               # DESENVOLVIDO POR JOHN HEBERTY DE FREITAS - FUNÇÕES AUXILIARES
from glob import glob
import pandas as pd
import numpy as np
import pickle
import os
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import math
import json

from Modulos.BROWSER.Engine import *
from Functions import *
from Script1 import *

%matplotlib inline

# VARIAVEIS PADRÃO

In [19]:
Direcao_nome        = "DIRECAO"             # NOME COMUM ENTRE OS BANCOS PARA O RUMO DO VENTO
Vento_nome          = "VENTO"               # NOME COMUM ENTRE OS BANCOS PARA O VENTO
Tempo_Analise       = 10                    # TEMPO EM ANOS PARA FAZER A ANALISE
Tempo_Busca         = 3                     # MESES - DESCONTINUADO
MtoNo               = 1.944                 # RAZÃO DE METROS/S PARA NÓS/S
CasasDescimais      = 3                     # ARREDONDADMENTO EM 3 CASAS DESCIMAIS
RosadosVentos       = 16                     # QUANTIDADE DE BARRAS QUE RODA DOS VENTOS TERA
limites             = [3,   13, 20, 25, 40] # Limites para a Rosas dos Ventos
nomes_setores       = {
     4: ['N', 'W', 'S', 'E'],
     8: ['N', 'NW', 'W', 'SW', 'S', 'SE', 'E', 'NE'],
    16: ['N', 'NNW', 'NW', 'WNW', 'W', 'WSW', 'SW', 'SSW', 'S', 'SSE', 'SE', 'ESE', 'E', 'ENE', 'NE', 'NNE']
}


# VARIAVEIS PARA OTIMIZAÇÃO DE PISTA
largura             = 1920
altura              = 1080
espessura           = 2
tamanho_fonte       = 2
grau_meio_setor     = (360/RosadosVentos)/2 
fonte               = cv.FONT_HERSHEY_SIMPLEX
make_video          = False
pasta_imagens       = 'DADOS\\IMGS'
caminho_saida_video = 'RunwayOrientation.mp4'

# ARQUIVOS PARA ANALISE
estacoes_convencionais = glob(os.path.join("Dados", "INPUT", "CONVENCIONAIS", "*.csv"))
estacoes_automaticas = glob(os.path.join("Dados", "INPUT", "AUTOMATICAS", "*.csv"))

# ABRINDO O DADO DOS VENTOS

In [20]:
def ReadDataSets(paths):
    
    pd.set_option('future.no_silent_downcasting', True)
    
    DataFiles = {}
    count = 1
    
    # BASE PATH
    caminho_arquivo = 'DataFiles.pickle'
    path = os.path.commonpath(paths).replace("INPUT", "TREATED")
    path_pickle = os.path.join(path, caminho_arquivo)

    # SE O PICKLE NÃO EXISTIR
    if os.path.exists(path_pickle) is False:
        
        # PERCORRENDO OS PATHS
        for filePath in paths:
            print(f"READ PATH: {filePath}")
            with open(filePath,"r") as file:
                text_lines = file.readlines()
                Name = [row.strip() for row in text_lines[0].strip().split("Nome:") if row != ""][0]
                Latitude = [row.strip() for row in text_lines[2].strip().split("Latitude:") if row != ""][0]
                Longitude = [row.strip() for row in text_lines[3].strip().split("Longitude:") if row != ""][0]
                Altitude = [row.strip() for row in text_lines[4].strip().split("Altitude:") if row != ""][0]
                title = text_lines[10].strip()[:-1].split(";")
                dataset = text_lines[11:]
                dataset = [line.strip()[:-1].split(";") for line in dataset]
                dataset = [line for line in dataset if line[0] != ""]
                dataset = [{title[index]:line[index] for index in range(len(title))} for line in dataset]
                df = pd.DataFrame(dataset)
                df["DATA"] = (df[df.columns[0]] + " " + df[df.columns[1]]).apply(lambda x: str(x).strip())
                datas = []
                for data in df["DATA"].values:
                    if ":00:00" in data:
                        data = data.replace(":00:00",":00")
                    if len(data.split()[1].split(":")[0].strip())==1:
                        data = data.replace(" "," 0")
                    datas.append(data)
                df["DATA"] = datas
                formats = ["%Y-%m-%d %H%M", "%d/%m/%Y %H:%M"]
                for format in formats:
                    try:
                        df["DATA"] = pd.to_datetime(df["DATA"], format=format)
                    except Exception as e:
                        print(e)
                # list([row for row in df.columns if "TEMPERATURA ORVALHO MAX" in row]) + \
                # list([row for row in df.columns if "TEMPERATURA ORVALHO MIN" in row]) + \
                # list([row for row in df.columns if "PRESSAO ATMOSFERICA MAX" in row]) + \
                # list([row for row in df.columns if "PRESSAO ATMOSFERICA MIN" in row]) + \
                columns = ["DATA"] + list([row for row in df.columns if "UMIDADE RELATIVA" in row])
                #columns = columns  + list([row for row in df.columns if "TEMPERATURA DO PONTO DE ORVALHO" in row])
                # columns = columns  + list([row for row in df.columns if "TEMPERATURA MINIMA" in row])
                # columns = columns  + list([row for row in df.columns if "TEMPERATURA MAXIMA" in row])
                columns = columns  + list([row for row in df.columns if "BULBO SECO" in row])
                # columns = columns  + list([row for row in df.columns if "PRESSAO" in row and "ESTACAO" in row])
                # columns = columns  + list([row for row in df.columns if "PRESSAO" in row and "MAR" in row])
                DIRECAO = list([row for row in df.columns if "DIRECAO" in row])
                columns = columns  + DIRECAO
                VENTO_MAX_AUTO = list([row for row in df.columns if ("VENTO" in row and "RAJADA" in row)])
                VENTO_NORM = list([row for row in df.columns if ("VENTO" in row and "VELOCIDADE" in row and "RAJADA" not in row)])
                VENTO = (VENTO_MAX_AUTO if len(VENTO_MAX_AUTO) > 0 else VENTO_NORM)
                columns = columns + VENTO
                df = df[columns].copy()
                for column in [row for row in df.columns if "DATA" not in row]:
                    df[column] = df[column].replace(["None", "none", "null", "Null", ""], np.nan).infer_objects(copy=False)
                # DROPANDO VALORES NULOS NO VENTO E DIRECAO
                df.loc[df[VENTO + DIRECAO].dropna(axis=0).index]
                df = df.reset_index(drop=True)
                df.columns = [
                    "DATA",
                    f"UMIDADE RELATIVA{count}",
                    #f"TEMPERATURA DO PONTO DE ORVALHO{count}",
                    # f"TEMPERATURA MINIMA{count}",
                    # f"TEMPERATURA MAXIMA{count}",
                    # f"TEMPERATURA ORVALHO MAX{count}",
                    # f"TEMPERATURA ORVALHO MIN{count}",
                    f"BULBO SECO{count}",
                    # f"PRESSAO ATMOSFERICA MAX{count}",
                    # f"PRESSAO ATMOSFERICA MIN{count}",
                    #f"PRESSAO AO NIVEL DA ESTACAO{count}",
                    #f"PRESSAO AO NIVEL DO MAR{count}",
                    f"DIRECAO{count}",
                    f"VENTO{count}",
                ]
                def Convert(x):
                    x = str(x).replace(",",".")
                    try:
                        return float(x)
                    except Exception as e:
                        return x
                for column in [row for row in df.columns if "DATA" not in row]:
                    df[column] = df[column].apply(lambda x: Convert(x))
                    df[column] = df[column].apply(lambda x: round(x, CasasDescimais))
                df = df.drop_duplicates()
                # ELIMINANDO VENTOS ZERADOS
                df = df[df[df.columns[-1]]>0]
                # TRANSFORMANDO M/S PARA NÓS/S
                df[df.columns[-1]] = df[df.columns[-1]].apply(lambda x:  round(x * MtoNo, CasasDescimais))
                df = df.reset_index(drop=True)
                DataFiles[Name] = {
                    "Local": (eval(Latitude), eval(Longitude)), 
                    "Altidude": Altitude, 
                    "Data": df,
                    "Name File": os.path.basename(filePath),
                    "Path File": filePath
                }
            count+=1

        # SE EXISTIR DADOS SALVA O PICKLE
        if DataFiles != {}:
            with open(path_pickle, 'wb') as arquivo:
                pickle.dump(DataFiles, arquivo)
    
    # SE EXISTIR PEGUE DO HISTORICO
    else:
        
        # Carregar o dicionário do arquivo usando pickle
        with open(path_pickle, 'rb') as arquivo:
            DataFiles = pickle.load(arquivo)
    
    return DataFiles

# DataFilesNoAuto = ReadDataSets(estacoes_convencionais)
DataFilesAuto = ReadDataSets(estacoes_automaticas)

NomeDadosAtual = "GUARULHOS"
tabelao = DataFilesAuto[NomeDadosAtual]["Data"].copy() # TRAMANDAI
# tabelao[[row for row in tabelao.columns if "DIRECAO" in row]].value_counts().to_frame()

RECORTANDO DADOS DE TEMPERATURA MENSAL

In [21]:
count = 1
all_temp_max = []
for aeroporto in DataFilesAuto:
    info = DataFilesAuto[aeroporto]
    DF = info["Data"].copy()
    DF = DF[DF["DATA"]>=DF["DATA"].max()-timedelta(days=365*20)]
    DF["YEAR"] = DF["DATA"].apply(lambda x: x.year)
    DF = DF.sort_values(["DATA"]).reset_index(drop=True)
    
    DF["YEAR"] = DF["DATA"].apply(lambda x: x.year)
    groupby = DF.groupby("YEAR")[f"BULBO SECO{count}"].mean()
    TEMP_MAX_groupby = groupby.max()
    YEAR_FIND = groupby.to_frame()[groupby.to_frame()[f"BULBO SECO{count}"] == TEMP_MAX_groupby].index[0]
    
    DF["MONTH"] = DF["DATA"].apply(lambda x: x.month)
    groupby = DF[(DF["YEAR"]==YEAR_FIND)].groupby("MONTH")[f"BULBO SECO{count}"].mean()
    TEMP_MAX_groupby = groupby.max()
    MONTH_FIND = groupby.to_frame()[groupby.to_frame()[f"BULBO SECO{count}"] == TEMP_MAX_groupby].index[0]
        
    DF_FIND = DF[(DF["YEAR"]==YEAR_FIND) & (DF["MONTH"]==MONTH_FIND)]
    TEMP_MAX = DF_FIND[f"BULBO SECO{count}"].max()
    DATA_TEMP_MAX = DF_FIND[DF_FIND[f"BULBO SECO{count}"]==TEMP_MAX]
    
    year = DATA_TEMP_MAX["DATA"].iloc[-1].year
    month = DATA_TEMP_MAX["DATA"].iloc[-1].month
    data = datetime(year + 1 if month == 12 else year, month + 1 if month < 12 else 1,1, 23, 59, 59) - timedelta(days=1)
    filtro1 = DF[(DF["DATA"]>=datetime(year,month,1)) & (DF["DATA"]<=data)].copy()
    filtro2 = filtro1[filtro1[f"BULBO SECO{count}"]>filtro1[f"BULBO SECO{count}"].mean()+filtro1[f"BULBO SECO{count}"].std()*0.5]
    TEMP_MAX_MEAN = round(filtro2[f"BULBO SECO{count}"].mean(), 3)
    
    if str(TEMP_MAX_MEAN)[0].isdigit() is False: break
    
    # if aeroporto == "ITAJAI": break
    all_temp_max.append({
        "AEROPORTO": aeroporto,
        "TEMP_MAX": TEMP_MAX,
        "TEMP_MEAN": TEMP_MAX_MEAN,
        "MOUNTH": month,
        "YEAR": year
    })
    
    count+=1
all_temp_max = pd.DataFrame(all_temp_max)
all_temp_max

,AEROPORTO,TEMP_MAX,TEMP_MEAN,MOUNTH,YEAR
0,BRASILIA,30.3,26.841,3,2024
1,MACAPA,30.9,30.900,7,2015
2,FORTALEZA,33.0,31.520,3,2010
3,JOAO PESSOA,34.0,31.667,3,2024
4,FLORIANOPOLIS,34.3,30.600,2,2024
5,CURITIBA,31.7,27.190,2,2024
6,PORTO ALEGRE- BELEM NOVO,35.6,29.761,2,2024
7,GUARULHOS,36.3,30.272,1,2015
8,RECIFE (CURADO),32.0,30.571,2,2016
9,MACEIO,34.0,31.088,3,2024


# BAIXANDO OS DADOS DOS PERIODOS QUE O EL NINO E LA NINA OCORREU HISTORICAMENTE

In [22]:
link_table_elnino_lanina = "https://origin.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/ONI_v5.php"

req = get(link_table_elnino_lanina)

def Table_ElNino_LaNina(table:list):
    # Iterar sobre as tabelas encontradas
    table = tables[0]
    
    # Extrair os dados da tabela
    data = []
    dataColor = []
    for row in table.find_all('tr'):
        values = []
        valuesColor = []
        # RECORTANDO CORES E VALORES
        for cell in row.find_all(['th', 'td']):
            filho = list(cell.children)[0]
            try:
                style = filho.get("style")
            except Exception as e:
                style = None
            color = 0
            if style != None:
                color = style.split("color:")[-1].strip()
            values.append(cell.text.strip())
            valuesColor.append(color)
        data.append(values)
        dataColor.append(valuesColor)

    # Converter para DataFrame do Pandas
    TEMEPRATURAS_MENSAIS = pd.DataFrame(data[1:], columns=data[0])
    TEMEPRATURAS_MENSAIS.columns = ["Year"] + list(range(1,13))
    # TEMEPRATURAS_MENSAIS = TEMEPRATURAS_MENSAIS[TEMEPRATURAS_MENSAIS["Year"]!="Year"]
    # TEMEPRATURAS_MENSAIS = TEMEPRATURAS_MENSAIS[TEMEPRATURAS_MENSAIS["Year"].apply(lambda x: str(x).strip()[0].isdigit())]
    TEMEPRATURAS_MENSAIS = TEMEPRATURAS_MENSAIS.reset_index(drop=True)
    DF_ELINO_LANINA = pd.DataFrame(dataColor[1:], columns=data[0])
    DF_ELINO_LANINA[data[0][0]] = TEMEPRATURAS_MENSAIS[data[0][0]]
    DF_ELINO_LANINA.columns = ["Year"] + list(range(1,13))
    DF_ELINO_LANINA = DF_ELINO_LANINA.reset_index(drop=True)
    
    return TEMEPRATURAS_MENSAIS, DF_ELINO_LANINA

if req.ok:
    # Use StringIO para envolver a string HTML
    soup = BeautifulSoup(req.text, 'html.parser')
    # Encontrar todas as tabelas
    attrs = {'align': 'center', 'border': '1'}
    tables = soup.find_all('table', attrs=attrs)

    TEMEPRATURAS_MENSAIS, DF_ELINO_LANINA = Table_ElNino_LaNina(tables)

In [23]:
times_events = []
for index, row in DF_ELINO_LANINA.iterrows():
    Year = eval(str(row.Year))
    try:
        month_blue         = row[row == "blue"].index.values
        min_month_blue     = datetime(year=Year, month=month_blue.min(), day=1)
        max_month_blue     = datetime(year=Year, month=month_blue.max(), day=1)
    except Exception as e:
        min_month_blue      = np.nan
        max_month_blue      = np.nan
    
    try:
        month_red          = row[row == "red"].index.values
        min_month_red      = datetime(year=Year, month=month_red.min(), day=1)
        max_month_red      = datetime(year=Year, month=month_red.max(), day=1)
    except Exception as e:
        min_month_red      = np.nan
        max_month_red      = np.nan
        
    try:
        month_black        = row[row == "black"].index.values
        min_month_black    = datetime(year=Year, month=month_black.min(), day=1)
        max_month_black    = datetime(year=Year, month=month_black.max(), day=1)
    except Exception as e:
        min_month_black      = np.nan
        max_month_black      = np.nan
    
    times_events.append({
        "Year": Year,
        "BlueStart": min_month_blue,
        "BlueEnd": max_month_blue,
        "RedStart": min_month_red,
        "RedEnd": max_month_red,
        "BlackStart": min_month_black,
        "BlackEnd": max_month_black
    })

DF_PERIODOS_ELNINO_LANINA = pd.DataFrame(times_events)
DF_PERIODOS_ELNINO_LANINA.head()
# DF_PERIODOS_ELNINO_LANINA.to_csv("PeriodosElNinoLaNina.csv", encoding="UTF-8", sep=";", decimal=",")
# TEMEPRATURAS_MENSAIS.to_csv("TemperaturasMensais.csv", encoding="UTF-8", sep=";", decimal=",")

,Year,BlueStart,BlueEnd,RedStart,RedEnd,BlackStart,BlackEnd
0,1950,1950-01-01,1950-07-01,NaT,NaT,1950-08-01,1950-12-01
1,1951,NaT,NaT,1951-06-01,1951-12-01,1951-01-01,1951-05-01
2,1952,NaT,NaT,1952-01-01,1952-01-01,1952-02-01,1952-12-01
3,1953,NaT,NaT,1953-02-01,1953-12-01,1953-01-01,1953-01-01
4,1954,1954-05-01,1954-12-01,1954-01-01,1954-02-01,1954-03-01,1954-04-01


# BUSCANDO RUMO QUE MÁXIMIZA O FO

In [27]:
# ABRINDO NAVEGADOR PARA OBTER A DECLINAÇÃO MAGNETICA
url = 'https://ngdc.noaa.gov/geomag/calculators/magcalc.shtml'
driver = OpenBrowser(url=url)

# PERCORRENDO OS DADOS DE CADA AEROPORTOE OTIMIZANDO VALORES
TabelaFinal = []
for aeroporto, ideaeroporto in zip(DataFilesAuto, range(1,len(list(DataFilesAuto.keys()))+1)):

    print(f"AEROPORTO: {aeroporto} - #/$".replace("#",str(ideaeroporto)).replace("$",str(len(list(DataFilesAuto)))))
    
    # PEGANDO ULTIMOS 10 ANOS DE INFORMAÇÃO
    tabelao = DataFilesAuto[aeroporto]["Data"].copy()
    local = DataFilesAuto[aeroporto]["Local"]
    
    # OBTENDO A DECLINAÇÃO MAGNETICA
    MagneticDeclination = GetMagneticDeclination(local[0], local[1], driver)
    
    # LIMPANDO A PASTA DE IMGS SE FOR FAZER UM NOVO VIDEO
    if make_video is True:
        ClearFolder(pasta_imagens)

    # IMAGEM EM BRANCO
    imagem_branca = np.zeros((altura, largura, 3), dtype=np.uint8)

    # Calculando as coordenadas do centro da imagem
    centro_x = int(largura / 2)
    centro_y = int(altura / 2)

    maior = max(limites)
    menor = min(limites)

    proporcao = 0.24/maior
    comprimento = int(largura * (maior * proporcao))
    r_20 = 0
    for limite in limites[::-1]:
        r = int(largura * (limite * proporcao))
        if limite == limites[-1]:
            cv.circle(imagem_branca, (centro_x, centro_y), r, (255, 255, 255), -1)
        cv.circle(imagem_branca, (centro_x, centro_y), r, (0, 0, 255), 2)
        if limite == 20:
            r_20 = r

    # FILTRANDO DADOS PARA ANALISES
    if True:

        # FORÇANDO ORDENAÇÃO POR DATA
        tabelao = tabelao.sort_values(["DATA"]).reset_index(drop=True)

        # FILTRANDO O BANCO DE DADOS SOMENTE EM DADOS NEUTROS, EL NINO E LA NINA
        DFS_ELNINO_LANINA = {
            "DF_BLUE": pd.DataFrame(),
            "DF_RED": pd.DataFrame(),
            "DF_BLACK": pd.DataFrame(),
        }
        for index, row in DF_PERIODOS_ELNINO_LANINA.iterrows():
            if row.BlueStart != np.nan and row.BlueEnd != np.nan:
                Blue    = tabelao[(tabelao["DATA"]>=row.BlueStart) & (tabelao["DATA"]<=row.BlueEnd)]
                DFS_ELNINO_LANINA["DF_BLUE"] = pd.concat([DFS_ELNINO_LANINA["DF_BLUE"], Blue], axis=0, ignore_index=True)
            if row.RedStart != np.nan and row.RedEnd != np.nan:
                Red     = tabelao[(tabelao["DATA"]>=row.RedStart) & (tabelao["DATA"]<=row.RedEnd)]
                DFS_ELNINO_LANINA["DF_RED"] = pd.concat([DFS_ELNINO_LANINA["DF_RED"], Red], axis=0, ignore_index=True)
            if row.BlackStart != np.nan and row.BlackEnd != np.nan:
                Black   = tabelao[(tabelao["DATA"]>=row.BlackStart) & (tabelao["DATA"]<=row.BlackEnd)]
                DFS_ELNINO_LANINA["DF_BLACK"] = pd.concat([DFS_ELNINO_LANINA["DF_BLACK"], Black], axis=0, ignore_index=True)

        # CRIANDO DATAFRAMES COM MESMA QUANTIDADE DE LINHAS
        INDEXS = []
        for DF_NAME in DFS_ELNINO_LANINA:
            DF = DFS_ELNINO_LANINA[DF_NAME]
            INDEXS.append(DF.index.values[-1])
        for DF_NAME in DFS_ELNINO_LANINA:
            DF = DFS_ELNINO_LANINA[DF_NAME]
            DFS_ELNINO_LANINA[DF_NAME] = DF.iloc[-min(INDEXS):].reset_index(drop=True)
        
        # CONTABILIZANDO VENTOS PARA CADA DIRECAO EM PORCENTAGEM
        tabelao5 = tabelao[tabelao["DATA"]>=tabelao["DATA"].max()-timedelta(days=365*int(Tempo_Analise/2))].reset_index(drop=True).copy()
        tabelao10 = tabelao[tabelao["DATA"]>=tabelao["DATA"].max()-timedelta(days=365*Tempo_Analise)].reset_index(drop=True).copy()
        df_pct_ventos, name_setores = WindTablePCT(tabelao, RosadosVentos, Direcao_nome=Direcao_nome, Vento_nome=Vento_nome, limites=limites, save=False)
        
    # RETAS
    if True:
        
        graus_rumo = calcular_setores(RosadosVentos, name_setores)
        
        # Desenhando as linhas radiais para cada direção
        for name, angulo in graus_rumo.items():
            angulo = angulo[0]
            DrawRadialLine(imagem_branca, (centro_x, centro_y), comprimento, angulo, (0, 0, 0), 2)

    # IDENTIFICANDO CADA AREA DA ROSA DOS VENTOS COM UMA COR ESPECIFICA
    if True:
        imagem_AREA = imagem_branca.copy()

        # Crie uma máscara para identificar as regiões brancas na imagem
        branco_baixo = np.array([255, 255, 255])
        branco_alto = np.array([255, 255, 255])
        mascara_branco = cv.inRange(imagem_AREA, branco_baixo, branco_alto)

        # Encontre os contornos na imagem mascarada
        contornos, _ = cv.findContours(mascara_branco, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

        # REALIZANDO O AGRUPAMENTO DAS AREAS
        rotulos_clusters, grupos, center_grups = Agroup(contornos)

        # Criar uma nova imagem para desenhar os contornos coloridos
        cores_grupos = GenerateUniqueGrayColors(len(limites)*RosadosVentos)

        # SALVANDO INFORMAÇÕES SOBRE AS AREAS
        dict_area_info = {}

        # Para cada grupo de contornos, criar uma paleta de cores única
        contagem = 0
        
        # ORDENANDO DOS MAIORES GRUPOS PARA OS MENORES 
        grupos = sorted(grupos, key=lambda x: cv.contourArea(x[0]), reverse=True)
        id_grupo = 1
        for grupo, limite in zip(grupos, limites[::-1]):
            # Gerar cores únicas para o grupo com base no número de contornos
            parcial = {}
            for contorno in grupo:
                cor = cores_grupos[contagem]  # Selecionar uma cor única da paleta para cada área dentro do grupo
                cv.drawContours(imagem_AREA, [contorno], -1, cor, thickness=cv.FILLED)  # Preencher o contorno com a cor
        
                # # CALCULANDO ANGULO DO CENTRO PARA AREA PARA IDENTIFICAR O RUMO
                p1 = BaricentroArea(contorno)
                p2 = centro_x, centro_y
                angulo = CalculateAzimuth(p1, p2)
                rumo = ""
                for name, angulos in graus_rumo.items():
                    maior_ = max(angulos)
                    menor_ = min(angulos)
                    if angulo >= menor_ and angulo <= maior_:
                        rumo = name
                    elif angulo <= menor_ and angulo <= maior_:
                        rumo = "N"

                contagem+=1
                
                parcial[rumo] = {
                    # "CONTORNO": contorno, 
                    "AREA": cv.contourArea(contorno), 
                    "COR": cor,
                    "LIMITE": limite
                }
            dict_area_info[id_grupo] = parcial
            id_grupo+=1
    
    # DESENHANDO O RETANGULO DA PISTA E RECORTANDO
    if True:

        # Definindo as dimensões do retângulo
        largura_retangulo = r_20 * 2
        altura_retangulo = comprimento * 2

        # Calculando os vértices do retângulo
        vertices = np.array([
            [centro_x - largura_retangulo / 2, centro_y - altura_retangulo / 2],
            [centro_x + largura_retangulo / 2, centro_y - altura_retangulo / 2],
            [centro_x + largura_retangulo / 2, centro_y + altura_retangulo / 2],
            [centro_x - largura_retangulo / 2, centro_y + altura_retangulo / 2]
        ], dtype=np.float32)

        # CRIANDO PONTO DE RESTAURAÇÃO DA IMG SEM DESENHO
        imagem_AREA_BK = imagem_AREA.copy()
        
        # PERCORRENDO DADOS
        DFS = {
            ">=5"   :tabelao5,
            ">=10"  :tabelao10,
            "ELNINO":DFS_ELNINO_LANINA["DF_RED"],
            "LANINA":DFS_ELNINO_LANINA["DF_BLUE"],
            "NEUTRO":DFS_ELNINO_LANINA["DF_BLACK"],
        }
        dict_final = {}
        dict_pacial = {}
        for Name in DFS:
            Dataset = DFS[Name]
            
            # CONTABILIZANDO VENTOS PARA CADA DIRECAO EM PORCENTAGEM
            df_pct_ventos, name_setores = WindTablePCT(Dataset, RosadosVentos, Direcao_nome=Direcao_nome, Vento_nome=Vento_nome, limites=limites, save=False)
            
            # DADOS PARA SALVAR A OTIMIZAÇÃO
            FU_FINAL = 0
            FU_FINAL_MAXIMO = 0
            FU_FINAL_GRAU = 0
            
            # ANGULANDO A PISTA PARA BUSCAR OTIMIZAR O FU
            for grau in range(0, 181):
                
                # RESTAURANDO BK DA IMG SEM DESENHOS
                imagem_AREA = imagem_AREA_BK.copy()
                
                # Criando uma matriz de rotação de 30 graus em torno do centro da imagem
                theta = np.radians(grau)
                rot_mat = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
                centro_retangulo = np.array([centro_x, centro_y])
                vertices_rotacionados = np.dot(vertices - centro_retangulo, rot_mat.T) + centro_retangulo

                cor_pista = (255, 255, 255)
                # Desenhando o retângulo na imagem
                cv.polylines(imagem_AREA, [vertices_rotacionados.astype(np.int32)], isClosed=True, color=cor_pista, thickness=2)
                
                # Crie uma máscara para identificar as regiões brancas na imagem
                mascara = cv.inRange(imagem_AREA, cor_pista, cor_pista)

                # Encontre os contornos na imagem mascarada
                contornos, _ = cv.findContours(mascara, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
                
                # PINTANDO A PISTA COMPLETA DE UMA COR PARA IDENTIFICAR POR MASCARA E RECORTAR
                imagem_AREA2 = imagem_AREA_BK.copy()
                cv.drawContours(imagem_AREA2, contornos, -1, cor_pista, thickness=cv.FILLED)
                
                # Aplicar a máscara na imagem original
                mascara_recorte = cv.inRange(imagem_AREA2, cor_pista, cor_pista)
                pista_recortada = cv.bitwise_and(imagem_AREA, imagem_AREA, mask=mascara_recorte)
                
                # PUXANDO TABELA DE VENTOS DO SCRIPT 1 E RENOMEANDO AS ValoresS 
                # PARA UMA CONSULTA MAIS FACILITADA
                df_pct_ventos_cp = df_pct_ventos.copy()
                df_pct_ventos_cp.columns = limites + ["40+"]

                # FIXANDO A PISTA DE MAIOR THETA
                if FU_FINAL > FU_FINAL_MAXIMO:
                    FU_FINAL_MAXIMO = FU_FINAL
                    FU_FINAL_GRAU = grau
                
                # SE EXISTIR UM FU DESENHE 
                if FU_FINAL_MAXIMO > 0:
                            
                    # Criando uma matriz de rotação de 30 graus em torno do centro da imagem
                    theta = np.radians(FU_FINAL_GRAU)
                    rot_mat = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
                    centro_retangulo = np.array([centro_x, centro_y])
                    vertices_rotacionados = np.dot(vertices - centro_retangulo, rot_mat.T) + centro_retangulo
                    
                    cor_pista = (0, 255, 0)
                    # Desenhando o retângulo na imagem
                    cv.polylines(imagem_AREA, [vertices_rotacionados.astype(np.int32)], isClosed=True, color=cor_pista, thickness=2)

                    # DESENHANDO REFERENCIA DOS DADOS
                    DrawReferenceRUNWAY(imagem_AREA, (centro_x, centro_y), comprimento, FU_FINAL_GRAU, (255, 165, 0), 25)
                    
                # ANALISANDO O FATOR DE ULILIZAÇÃO DA PISTA ATUAL
                FU_FINAL = 0
                for id_grupo in dict_area_info:
                    grupo = dict_area_info[id_grupo]
                    for rumo in grupo:
                        Info = grupo[rumo]
                        cor_grupo = Info["COR"]
                        limite = Info["LIMITE"]
                        
                        # Crie uma máscara para identificar as regiões brancas na imagem
                        mascara = cv.inRange(pista_recortada.copy(), cor_grupo, cor_grupo)
                        
                        # Encontre os contornos na imagem mascarada
                        contorno, _ = cv.findContours(mascara, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
                        contorno = list(contorno)
                        
                        # NEM TODA AERA ESTA DENTRO DA PISTA LOGO 
                        if len(contorno) > 0:
                            contorno = contorno[0]
                            # IDENTIFICANDO AREA DO CONTORNO DENTRO DA PISTA
                            area_dentro_PPD = cv.contourArea(contorno)
                        
                        # CASO NÃO ESTIVER SETE ZERO
                        else:
                            area_dentro_PPD = 0
                        
                        # FAZENDO A FRAÇÃO DOS VENTROS DENTRO DA PISTA
                        fracao_dentro = (area_dentro_PPD / Info["AREA"]) * df_pct_ventos_cp.loc[rumo][limite]
                        
                        FU_FINAL = FU_FINAL + fracao_dentro
                FU_FINAL = round(FU_FINAL, 4)

                # INFORMAÇÕES SOBRE A PISTA ATUAL
                if True:
                
                    # INFORMAÇÕES SOBRE A PISTA ATUAL
                    texto = f'DIRECTION NOW' 
                    cv.putText(imagem_AREA, texto, (largura-465, 90), fonte, tamanho_fonte, (255, 255, 255), espessura, cv.LINE_AA)
                    
                    texto = f'FO: {FU_FINAL}%'
                    cv.putText(imagem_AREA, texto, (largura-465, 180), fonte, tamanho_fonte, (255, 255, 255), espessura, cv.LINE_AA)
                    
                    # pista = RunwayOrientation(grau)
                    texto = f'RUMO: {grau}*'
                    cv.putText(imagem_AREA, texto, (largura-465, 270), fonte, tamanho_fonte, (255, 255, 255), espessura, cv.LINE_AA)
                
                    # INFORMAÇÕES SOBRE OS VENTOS DE TRAVEZ ESQUERDO
                    texto = f'CROSS WIND: {1 - FU_FINAL}%'
                    cv.putText(imagem_AREA, texto, (largura-465, 360), fonte, tamanho_fonte, (255, 255, 255), espessura, cv.LINE_AA)
                
                # INFORMAÇÕES SOBRE A MELHOR PISTA
                if True:
                    cor_pista = (0, 255, 0)
                    
                    # INFORMAÇÕES SOBRE A PISTA ATUAL
                    texto = f'BEST DIRECTION'
                    cv.putText(imagem_AREA, texto, (40, 90), fonte, tamanho_fonte, cor_pista, espessura, cv.LINE_AA)
                    
                    texto = f'FO: {FU_FINAL_MAXIMO}%'
                    cv.putText(imagem_AREA, texto, (40, 180), fonte, tamanho_fonte, cor_pista, espessura, cv.LINE_AA)
                    
                    # pista = RunwayOrientation(FU_FINAL_GRAU)
                    texto = f'RUMO: {FU_FINAL_GRAU}*'
                    cv.putText(imagem_AREA, texto, (40, 270), fonte, tamanho_fonte, cor_pista, espessura, cv.LINE_AA)
                    
                    # INFORMAÇÕES SOBRE OS VENTOS DE TRAVEZ ESQUERDO
                    texto = f'CROSS WIND: {1 - FU_FINAL_MAXIMO}%'
                    cv.putText(imagem_AREA, texto, (40, 360), fonte, tamanho_fonte, cor_pista, espessura, cv.LINE_AA)
                
                # DESENHANDO REFERENCIA DOS DADOS
                DrawReferenceRUNWAY(imagem_AREA, (centro_x, centro_y), comprimento, grau, (255, 165, 0), 25)
                
                # INFORMAÇÕES SOBRE OS VENTOS DE TRAVEZ ESQUERDO
                texto = f'LOCAL: {NomeDadosAtual}'
                cv.putText(imagem_AREA, texto, (40, altura-45), fonte, int(tamanho_fonte/2), cor_pista, espessura, cv.LINE_AA)

                if make_video is True:
                    
                    # SALVANDO IMAGEM PARA O VIDEO FINAL
                    cv.imwrite(os.path.join("DADOS", "IMGS", f"IMG{grau}.jpg"), imagem_AREA)

            dict_pacial[Name] = {
                "FO": FU_FINAL_MAXIMO,
                "RUMO": FU_FINAL_GRAU,
                "CROSS WIND": 1 - FU_FINAL_MAXIMO,
            }
            
        dict_final[aeroporto] = dict_pacial
        TabelaFinal.append(dict_final)
        with open("TabelaFinal", "w", encoding="UTF-8") as file:
            json.dump(TabelaFinal, file)
        
    # SE ATIVADO CRIA O VIDEO
    if make_video is True:
        
        # Exemplo de uso da função
        CreateVideo(pasta_imagens, caminho_saida_video)
    
    # MOSTRANDO IMGS
    if False:

        # Converta de BGR para RGB (Matplotlib usa RGB)
        imagem_AREA_rgb = cv.cvtColor(imagem_AREA, cv.COLOR_BGR2RGB)

        # Crie uma figura e um eixo
        fig, ax = plt.subplots(figsize=(6, 6))
        # Mostre a imagem
        ax.imshow(imagem_AREA_rgb)
        # Remova os eixos
        ax.axis('off')
        # Exiba a imagem
        plt.show()

        # fig, axes = plt.subplots(1, 2, figsize=(10, 5))

        # # Plotando a primeira imagem no primeiro subplot
        # imagem_AREA_ = cv.cvtColor(imagem_AREA, cv.COLOR_BGR2RGB)
        # axes[0].imshow(imagem_AREA_)
        # axes[0].set_title('Rosewind with Runway')
        # axes[0].axis('off')

        # # Plotando a segunda imagem no segundo subplot
        # # pista_recortada_ = cv.cvtColor(pista_recortada, cv.COLOR_BGR2RGB)
        # # axes[1].imshow(pista_recortada_)
        # # axes[1].set_title('Runway Cutout')
        # # axes[1].axis('off')

        # # Mostrando os subplots
        # plt.tight_layout()
        # plt.show()


In [31]:
DataFilesAuto[aeroporto]["Local"]

(-15.78944444, -47.92583332)

In [29]:
ideaeroporto

1

# CRIANDO A WINDROSE

In [ ]:
# Criar uma figura e adicionar os eixos para a rosa-dos-ventos
figura = plt.figure(figsize=(12, 8))
rict_rectangle = [0, 0, 1, 1]
ax = WindroseAxes(figura, rict_rectangle)
figura.add_axes(ax)

# Definir os dados de direção e magnitude do vento
direcao = tabelao[[column for column in tabelao.columns if Direcao_nome in column][0]]  # Graus de direção do vento
magnetude = tabelao[[column for column in tabelao.columns if Vento_nome in column][0]]  # Magnitude do vento

# Definir algumas configurações opcionais para o gráfico
displayed = True
divisores_dados = [0,3,13,20,25,40] # bins (array 1D ou inteiro, opcional) – número de bins ou uma variável de sequência de bins. Se não definido, bins=6 entre min(var) e max(var).
blowto = False # blowto (bool, opcional.) – se True, a rosa-dos-ventos será pi girada, para mostrar para onde o vento sopra (útil para rosa poluente).
divisores_angular = 8 # Se não definido, nsetores=16, cada setor será 360/16=22,5° e a tabela computada resultante será alinhada com os pontos cardeais.
palette_color = [
    'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r',
    'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r',
    'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r',
    'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r',
    'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r',
    'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r',
    'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r',
    'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r',
    'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r',
    'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn',
    'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r',
    'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone',
    'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool',
    'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'crest', 'crest_r',
    'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'flare', 'flare_r', 'gist_earth',
    'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat', 'gist_heat_r', 'gist_ncar',
    'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg',
    'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'hot',
    'hot_r', 'hsv', 'hsv_r', 'icefire', 'icefire_r', 'inferno', 'inferno_r', 'jet', 'jet_r',
    'magma', 'magma_r', 'mako', 'mako_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r',
    'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'rocket',
    'rocket_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r', 'tab10',
    'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r', 'terrain',
    'terrain_r', 'turbo', 'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted',
    'twilight_shifted_r', 'viridis', 'viridis_r', 'vlag', 'vlag_r', 'winter', 'winter_r'
]
map_color = plt.get_cmap("Accent")
opening = 0.8 # ( float, opcional ) – entre 0,0 e 1,0, para controlar o espaço entre cada setor (1,0 para nenhum espaço)

# Criar o gráfico da rosa-dos-ventos
#ax.contour(direcao, magnetude, normed=displayed, bins=divisores_dados, nsector=divisores_angular, cmap=map_color)#, opening=opening)
ax.bar(direcao, magnetude, normed=displayed, bins=divisores_dados, blowto=blowto, nsector=divisores_angular, cmap=map_color, opening=opening)

# Configurar os eixos e legendas
ax.tick_params(axis="both", which="major", labelsize=20)  # Configurar os tamanhos das fontes nos eixos
ax.legend(loc="center left", decimal_places=1, bbox_to_anchor=(1.12, 0.5), fontsize=20)  # Adicionar uma legenda ao gráfico
ax.set_rlabel_position(125)  # Posição dos rótulos dos eixos radiais
ax.yaxis.set_major_formatter(tkr.FormatStrFormatter("%2.2f"))  # Formato dos rótulos dos eixos radiais
